In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import p_power as pp
import torch as tc
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from keras import Input, optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import L1, L2, L1L2, OrthogonalRegularizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate
from sklearn import preprocessing, linear_model, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


1. Read in Data

In [48]:
df = pd.read_csv('../data/l_pi.csv')


In [49]:
print(df.describe())


         velocity  weightpercentw  diametermm  thicknessmm    heightin  \
count  256.000000      256.000000  256.000000        256.0  256.000000   
mean     4.447500        0.375000    0.020000          0.0   43.750000   
std      1.421588        0.365578    0.007085          0.0   25.636983   
min      2.550000        0.100000    0.010000          0.0   13.000000   
25%      3.487500        0.137500    0.017500          0.0   25.000000   
50%      4.445000        0.200000    0.020000          0.0   40.500000   
75%      5.405000        0.437500    0.022500          0.0   59.250000   
max      6.350000        1.000000    0.030000          0.0   81.000000   

       craterdiameterfromouteredgesmm  craterdiameterfromouteredgesmmno  \
count                      256.000000                        256.000000   
mean                        53.556250                         64.258594   
std                         21.430322                         25.712146   
min                         13.50

2. Separate Column Paramters into separate int pd vects and scalars

In [50]:
out_full = df.iloc[:, 0]
vel = df.iloc[:, 2]
wp = df.iloc[:, 3]
d_mm = df.iloc[:, 4]
t_mm = df.iloc[:, 5]
sigma = df.iloc[:, 24]
nu = df.iloc[:, 25]
# pi1 = pd.to_numeric(df.iloc[:, 29], errors='coerce')
# pi2 = pd.to_numeric(df.iloc[:, 30], errors='coerce')
# pi3 = pd.to_numeric(df.iloc[:, 31], errors='coerce')
# pi4 = pd.to_numeric(df.iloc[:, 32], errors='coerce')
# pi5 = pd.to_numeric(df.iloc[:, 33], errors='coerce')
# pi6 = pd.to_numeric(df.iloc[:, 34], errors='coerce')
# pi7 = pd.to_numeric(df.iloc[:, 35], errors='coerce')
# pi8 = pd.to_numeric(df.iloc[:, 36], errors='coerce')


out_trunk = df.iloc[:, 44]

pi1 = df.iloc[:, 29]
pi2 = df.iloc[:, 30]
pi3 = df.iloc[:, 31]
pi4 = df.iloc[:, 32]
pi5 = df.iloc[:, 33]
pi6 = df.iloc[:, 34]
pi7 = df.iloc[:, 35]
pi8 = df.iloc[:, 36]

print(out_trunk)

0                      Splash
1                      Splash
2                      Splash
3      Broken or Intact Sheet
4                      Splash
                ...          
251                      Lump
252                      Lump
253                      Lump
254                      Lump
255                      Lump
Name: newcat1, Length: 256, dtype: object


3. Concat pi groups together and the dimension paramters together

In [51]:
phi = tc.Tensor([[0, 0, 1, 1, 1], [-2, -2, -1, 1, 1], [2, 0, 0, 0, 0], [0, 2, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, -2, -2], [0, 0, 0, 2, 0], [0, 0, 0, 0, 2]])
phi_T = phi.mT
phi_G = phi_T @ phi
print(phi_G)

x_targ = tc.Tensor([[-.5], [0], [1], [.5], [-.5]])
x_start = tc.Tensor(tc.randn(5, 1))
print(x_start)


phi_norm, vect = pp.p_power(phi, p=2, s_max=100, type=tc.float)
print(phi_norm)
print(vect)


for i in range(100):
    start_norm = tc.linalg.vector_norm(x_start, ord=2, keepdims=True)
    x_start = (phi_G @ x_start) / start_norm
    x_start = pp.dual(x_start, p=2, dim=0, type=tc.float)
print(x_start) #+ 2.28892574923842)
print(start_norm)

res = phi @ x_start
print(res)

res_norm = tc.linalg.vector_norm(res, ord=2, keepdims=True)

print(res_norm)

tensor([[ 8.,  4.,  2., -2., -2.],
        [ 4.,  8.,  2., -2., -2.],
        [ 2.,  2.,  6.,  0.,  0.],
        [-2., -2.,  0., 10.,  6.],
        [-2., -2.,  0.,  6., 10.]])
tensor([[ 0.0606],
        [-1.6095],
        [ 1.1397],
        [ 0.1158],
        [ 0.4233]])
tensor([[[4.3193]]])
tensor([[[ 0.3885],
         [ 0.3884],
         [ 0.1229],
         [-0.5844],
         [-0.5844]]])
tensor([[-0.3882],
        [-0.3882],
        [-0.1227],
        [ 0.5846],
        [ 0.5846]])
tensor([[1.]])
tensor([[ 1.0465],
        [ 2.8447],
        [-0.7764],
        [-0.7764],
        [-0.2454],
        [-2.3385],
        [ 1.1692],
        [ 1.1692]])
tensor([[4.3193]])


In [52]:
params = pd.concat([vel, wp, d_mm, t_mm, sigma, nu], axis=1)
# print(params)

tensor_params = tc.tensor(params.values, dtype=tc.float32)

phi_plus = tc.concat([phi, res], dim=1)
print(phi_plus.shape)
# pi_alg = (np.log(tensor_params)) @ 

torch.Size([8, 6])


In [77]:
pi_groups = pd.concat([pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8], axis=1)



# pi_groups = pd.concat([pi1, pi2, pi3, pi4, pi5], axis=1)

print(pi_groups.dtypes)


print(pi_groups)

pi1    float64
pi2    float64
pi3    float64
pi4    float64
pi5    float64
pi6    float64
pi7    float64
pi8    float64
dtype: object
              pi1           pi2          pi3        pi4       pi5  \
0    50991.962110  50541.334112  3105.712433  24.579584  0.061449   
1     5766.579789   5615.703790  3105.712433  24.579584  0.553041   
2     1479.863947   1403.925948  3105.712433  24.579584  2.212163   
3      846.911845    789.708345  3105.712433  24.579584  3.932733   
4    50991.962110  50541.334112  3105.712433  55.304064  0.061449   
..            ...           ...          ...        ...       ...   
251    185.134432    158.921609    61.240388   9.633741  0.385350   
252  10373.685582  10170.982999    61.240388  15.052721  0.006021   
253   1198.343416   1130.109222    61.240388  15.052721  0.054190   
254    317.144403    282.527306    61.240388  15.052721  0.216759   
255    185.134432    158.921609    61.240388  15.052721  0.385350   

              pi6         pi7       p

4. Create the Model

In [78]:
def train_and_predict_using_model(model_name = "", model =None):
    model.fit(X_train, Y_train)
    Y_pred_train = model.predict(X_train)
    cm_train = confusion_matrix(Y_train, Y_pred_train)
    print(model_name)
    print("====================================")
    print("Training Confusion Matrix: ")
    print(cm_train)
    acc_train = accuracy_score(Y_train, Y_pred_train)
    
    print("Training Accuracy: %.2f%%" % (acc_train*100))
    print("====================================")
    
    Y_pred = model.predict(X_test)
    cm_test = confusion_matrix(Y_test, Y_pred)
    print("Testing Confusion Matrix: ")
    print(cm_test)
    acc_test = acc_train = accuracy_score(Y_test, Y_pred)
    
    print("Testing Accuracy: %.2f%%" % (acc_test*100))
    print("====================================")

In [79]:
def train_and_predict_using_pimodel(model_name = "", model =None):
    model.fit(scaled_X_train_pi, Y_train_pi)
    Y_pred_train_pi = model.predict(scaled_X_train_pi)
    cm_train_pi = confusion_matrix(Y_train_pi, Y_pred_train_pi)
    print(model_name)
    print("====================================")
    print("Training Confusion Matrix: ")
    print(cm_train_pi)
    acc_train = (np.trace(cm_train_pi)) / np.sum(np.sum(cm_train_pi))
    
    print("Training Accuracy: %.2f%%" % (acc_train*100))
    print("====================================")
    
    Y_pred_pi = model.predict(scaled_X_test_pi)
    cm_test_pi = confusion_matrix(Y_test_pi, Y_pred_pi)
    print("Testing Confusion Matrix: ")
    print(cm_test_pi)
    acc_test = acc_train = np.trace(cm_test_pi) / np.sum(np.sum(cm_test_pi))
    
    print("Testing Accuracy: %.2f%%" % (acc_test*100))
    print("====================================")

In [80]:
X_train, X_test, Y_train, Y_test = train_test_split(params, out_trunk, test_size=0.2, random_state=42)
X_train_pi, X_test_pi, Y_train_pi, Y_test_pi = train_test_split(pi_groups, out_trunk, test_size=0.2, random_state=0)

# scaler = preprocessing.StandardScaler().fit(X_train)
# scaled_X_train = scaler.transform(X_train)
# scaled_X_test = scaler.transform(X_test)

scaler_pi = preprocessing.StandardScaler().fit(X_train_pi)
scaled_X_train_pi = scaler_pi.transform(X_train_pi)
scaled_X_test_pi = scaler_pi.transform(X_test_pi)
encoder = LabelEncoder()
encoder.fit(out_trunk)
encoded_ytest_pi = encoder.transform(Y_test_pi)
encoded_ytrain_pi = encoder.transform(Y_train_pi)
encoded_y_pi = encoder.transform(out_trunk)
# One-hot encode the target variable
encoded_ytrain_pi_onehot = to_categorical(encoded_ytrain_pi)

encoded_ytest_pi_onehot = to_categorical(encoded_ytest_pi)
encoded_y_pi_onehot = to_categorical(encoded_y_pi)

In [81]:

print(encoded_y_pi_onehot)



[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


5. Try some stuff


In [82]:
# Logistic Regression
linear_classifier = linear_model.LogisticRegression(random_state=123)
train_and_predict_using_model("Logistic Regression", linear_classifier)

Logistic Regression
Training Confusion Matrix: 
[[60  6  0 14]
 [11 13  0  0]
 [ 0  2 14  0]
 [23  0  0 61]]
Training Accuracy: 72.55%
Testing Confusion Matrix: 
[[10  4  0  6]
 [ 2  6  0  0]
 [ 0  1  2  0]
 [ 6  0  0 15]]
Testing Accuracy: 63.46%


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [83]:
linear_classifier_pi = linear_model.LogisticRegression()
train_and_predict_using_pimodel("Pi Logistic Regression", linear_classifier_pi)

Pi Logistic Regression
Training Confusion Matrix: 
[[69  2  0  7]
 [14  9  2  0]
 [ 2  4 10  0]
 [12  0  0 73]]
Training Accuracy: 78.92%
Testing Confusion Matrix: 
[[19  0  0  3]
 [ 6  1  0  0]
 [ 0  1  2  0]
 [ 2  0  0 18]]
Testing Accuracy: 76.92%


Decision Tree



In [84]:
# Decision Tree
decision_tree_clf = tree.DecisionTreeClassifier()
train_and_predict_using_model('Decision Tree', decision_tree_clf)

Decision Tree
Training Confusion Matrix: 
[[73  2  0  5]
 [ 9 15  0  0]
 [ 0  0 16  0]
 [24  0  0 60]]
Training Accuracy: 80.39%
Testing Confusion Matrix: 
[[12  3  0  5]
 [ 3  4  1  0]
 [ 0  0  3  0]
 [ 7  0  0 14]]
Testing Accuracy: 63.46%


In [85]:
# Decision Tree and Random Forests
decision_tree_clf_pi = tree.DecisionTreeClassifier()
train_and_predict_using_pimodel('Decision Tree', decision_tree_clf_pi)

Decision Tree
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[14  4  0  4]
 [ 2  5  0  0]
 [ 0  1  2  0]
 [ 5  0  0 15]]
Testing Accuracy: 69.23%


In [86]:
model4 = GradientBoostingClassifier(n_estimators=500, learning_rate=1.0, max_depth=6, random_state=357, loss='log_loss', criterion='squared_error', min_samples_split=5, min_samples_leaf=3, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0, init=None, subsample=1.0)
train_and_predict_using_pimodel('Gradient Boosting', model4)

# 93.515% accuracy

# model4 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.20, max_depth=6, random_state=0, loss='log_loss', criterion='squared_error', min_samples_split=3, min_samples_leaf=2, max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

# 93.75% accuracy

# model4 = GradientBoostingClassifier(n_estimators=250, learning_rate=1.255, max_depth=9, random_state=0, loss='log_loss', criterion='friedman_mse', min_samples_split=4, min_samples_leaf=2, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

# model4 = GradientBoostingClassifier(n_estimators=300, learning_rate=1.3, max_depth=12, random_state=357, loss='log_loss', criterion='friedman_mse', min_samples_split=4, min_samples_leaf=2, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0, init=None, subsample=1.0)
# train_and_predict_using_pimodel('Gradient Boosting', model4)

Gradient Boosting
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[18  1  0  3]
 [ 3  4  0  0]
 [ 0  0  3  0]
 [ 2  0  0 18]]
Testing Accuracy: 82.69%


Random Forest

In [90]:
forest = RandomForestClassifier(n_estimators=100, random_state=123, max_depth=8, max_features=6)
train_and_predict_using_model('Random Forest', forest)

Random Forest
Training Confusion Matrix: 
[[69  3  0  8]
 [ 8 16  0  0]
 [ 0  0 16  0]
 [21  0  0 63]]
Training Accuracy: 80.39%
Testing Confusion Matrix: 
[[ 9  4  0  7]
 [ 3  4  1  0]
 [ 0  0  3  0]
 [ 6  1  0 14]]
Testing Accuracy: 57.69%


In [93]:
forest = RandomForestClassifier(n_estimators=150, random_state=123, max_depth=12, max_features=12, criterion='entropy')

train_and_predict_using_pimodel('Random Forest', forest)

Random Forest
Training Confusion Matrix: 
[[78  0  0  0]
 [ 0 25  0  0]
 [ 0  0 16  0]
 [ 0  0  0 85]]
Training Accuracy: 100.00%
Testing Confusion Matrix: 
[[19  0  0  3]
 [ 0  7  0  0]
 [ 0  1  2  0]
 [ 1  0  0 19]]
Testing Accuracy: 90.38%


Neural Net


In [46]:
pi1_wghts = tf.exp(tf.constant([1.0, -1.0, 2.0, 0.0, 0.0, 0.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, -1.0, 2.0, 0.0, 0.0, 0.0])))
pi2_wghts = tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, -2.0, 0.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, -2.0, 0.0])))
pi3_wghts =tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, 0.0, 2.0])) / tf.linalg.norm(tf.exp(tf.constant([1.0, 1.0, 0.0, -2.0, 0.0, 2.0])))

pi_weights = tf.concat([[pi1_wghts], [pi2_wghts], [pi3_wghts]], 0)

print(pi_weights.shape)

(3, 6)


In [94]:
def train_and_predict_pinets(opt=None, model=None):
    
    if opt is None:
        model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    else:
    
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['categorical_accuracy'])
    
    model.fit(scaled_X_train_pi, encoded_ytrain_pi_onehot, epochs=200, batch_size=4, verbose=0)
    
    
    Y_pred_train_pi = model.predict(scaled_X_train_pi)

    # Evaluate the model
    scores = model.evaluate(scaled_X_train_pi, encoded_ytrain_pi_onehot)

    print("Neural Network Trainset: \n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

    Y_pred_pi = model.predict(scaled_X_test_pi)

    # Convert predictions to class labels
    Y_pred_pi_labels = np.argmax(Y_pred_pi, axis=1)
    Y_test_pi_labels = np.argmax(encoded_ytest_pi_onehot, axis=1)
    # import pdb; pdb.set_trace()
    cm_test = confusion_matrix(Y_test_pi_labels, Y_pred_pi_labels)
    print("Testing Confusion Matrix: ")
    print(cm_test)
    acc_test = accuracy_score(Y_test_pi_labels, Y_pred_pi_labels)

    print("Testing Accuracy: %.2f%%" % (acc_test * 100))
    print("====================================")
    
    print("Classification Report")
    print(classification_report(Y_test_pi_labels, Y_pred_pi_labels))
    
    print("====================================")
    
    print("Summary of Neural Network:")
    print(model.summary())
    
    print("====================================")
    
    return Y_test_pi_labels, Y_pred_pi_labels

    # print(Y_pred_pi)

In [127]:
# Neural Network
def create_pinet(comp = False):

    pi_net = Sequential()
   
    # 92.31% accuracy
    pi_net.add(Input(shape=(8,)))
    pi_net.add(Dense(16, kernel_regularizer=OrthogonalRegularizer(factor=0.02, mode="columns")))
    pi_net.add(Dense(32))
    pi_net.add(Dense(32, activation='silu'))
    pi_net.add(Dense(4, activation='softmax'))
    if comp is True:
        pi_net.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(), metrics=['categorical_accuracy'])
    
    return pi_net




In [128]:
pim = create_pinet(False)
train_and_predict_pinets(model=pim, opt=optimizers.Adam())



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - categorical_accuracy: 0.9857 - loss: 0.0692  
Neural Network Trainset: 
compile_metrics: 98.04%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Testing Confusion Matrix: 
[[18  1  0  3]
 [ 0  7  0  0]
 [ 0  1  2  0]
 [ 1  0  0 19]]
Testing Accuracy: 88.46%
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.78      1.00      0.88         7
           2       1.00      0.67      0.80         3
           3       0.86      0.95      0.90        20

    accuracy                           0.88        52
   macro avg       0.90      0.86      0.86        52
weighted avg       0.90      0.88      0.88        52

Summary of Neural Network:


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_57 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,630 (22.00 KB)

 Trainable params: 1,876 (7.33 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,754 (14.67 KB)

None


(array([3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 0, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 1, 3, 3, 1, 0, 3, 1, 2,
        3, 0, 0, 3, 3, 1, 3, 1]),
 array([3, 0, 0, 0, 0, 3, 1, 3, 0, 0, 3, 1, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3,
        2, 3, 3, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 1, 3, 3, 1, 0, 3, 1, 1,
        3, 3, 1, 3, 3, 1, 0, 1]))

In [162]:

silu_layer = pim.get_layer('silu').get_weights()
# print(silu_layer[0])
silu_layer_tensor = tc.complex(tc.tensor(silu_layer[0]).T, tc.zeros_like(tc.tensor(silu_layer[0]).T))
print(silu_layer_tensor.shape)
res, _ = pp.p_power(silu_layer_tensor, p=2)
print(res)


gelu_layer = pim.get_layer('gelu').get_weights()
# print(silu_layer[0])
gelu_layer_tensor = tc.complex(tc.tensor(gelu_layer[0]).T, tc.zeros_like(tc.tensor(gelu_layer[0]).T))
print(gelu_layer_tensor.shape)
res, _ = pp.p_power(gelu_layer_tensor, p=2)
print(res)

v = tc.linalg.vector_norm(gelu_layer_tensor[:,31], ord=2)
print('Gelu Col: ',v)

softmax_layer = pim.get_layer('softmax').get_weights()
# print(silu_layer[0])
softmax_layer_tensor = tc.complex(tc.tensor(softmax_layer[0]).T, tc.zeros_like(tc.tensor(softmax_layer[0]).T))
print(softmax_layer_tensor.shape)
res, _ = pp.p_power(softmax_layer_tensor, p=2)
print(res)

torch.Size([32, 6])
tensor([[[1.4339]]])
torch.Size([32, 64])
tensor([[[1.0320]]])
Gelu Col:  tensor(0.1545)
torch.Size([4, 32])
tensor([[[3.9600]]])


In [ ]:
# 90.625% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
# 92.19% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='gelu'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(16))
pi_net.add(Dense(4, activation='softmax')) """

In [733]:
"""  92.31% accuracy
    pi_net.add(Input(shape=(6,)))
    pi_net.add(Dense(32, activation='relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(64, activation='relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(32, activation='leaky_relu'))
    pi_net.add(Dense(64, activation='leaky_relu'))
    pi_net.add(Dropout(0.3))
    pi_net.add(Dense(128, activation='tanh'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(64, activation='relu'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(32, activation='gelu'))
    pi_net.add(Dense(4, activation='softmax')) """

"  92.31% accuracy\n    pi_net.add(Input(shape=(6,)))\n    pi_net.add(Dense(32, activation='relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(64, activation='relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(32, activation='leaky_relu'))\n    pi_net.add(Dense(64, activation='leaky_relu'))\n    pi_net.add(Dropout(0.3))\n    pi_net.add(Dense(128, activation='tanh'))\n    pi_net.add(Dropout(0.5))\n    pi_net.add(Dense(64, activation='relu'))\n    pi_net.add(Dropout(0.5))\n    pi_net.add(Dense(32, activation='gelu'))\n    pi_net.add(Dense(4, activation='softmax')) "

In [ ]:
# 93.75% accuracy
""" pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(64, activation='relu'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(32, activation='gelu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
""" # 94.23% accuracy
    pi_net.add(Input(shape=(6,)))
    pi_net.add(Dense(32, activation='silu'))
    pi_net.add(Dense(64, activation='silu'))
    pi_net.add(Dense(128, activation='tanh'))
    pi_net.add(Dropout(0.25))
    pi_net.add(Dense(256, activation='leaky_relu'))
    pi_net.add(Dense(512, activation='silu'))
    pi_net.add(Dropout(0.5))
    pi_net.add(Dense(64, activation='leaky_relu'))
    pi_net.add(Dropout(0.25))
    pi_net.add(Dense(32, activation='gelu'))
    pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
#  # 94.23% accuracy
#     pi_net.add(Input(shape=(8,)))
#     # pi_net.add(Dense(32, activation='silu'))
#     pi_net.add(Dense(64, activation='silu'))
#     pi_net.add(Dropout(0.35))
#     pi_net.add(Dense(128, activation='tanh'))
#     pi_net.add(Dropout(0.5))
#     pi_net.add(Dense(64, activation='elu'))
    
#     # pi_net.add(Dense(32, activation='gelu'))
#     pi_net.add(Dense(4, activation='softmax'))

In [ ]:
# # 94.23% accuracy more consistent
#     pi_net = Sequential()
   
#     # 94.23% accuracy
#     pi_net.add(Input(shape=(6,)))
#     # pi_net.add(Dense(32, activation='silu'))
#     pi_net.add(Dense(64, activation='silu'))
#     pi_net.add(Dropout(0.35))
#     pi_net.add(Dense(128, activation='tanh'))
#     pi_net.add(Dropout(0.5))
#     pi_net.add(Dense(64, activation='elu'))
    
#     # pi_net.add(Dense(32, activation='gelu'))
#     pi_net.add(Dense(4, activation='softmax'))

In [ ]:
    # # 94.23% accuracy
    # pi_net.add(Input(shape=(6,)))
    # pi_net.add(Dense(32, activation='silu'))
    # pi_net.add(Dense(64, activation='tanh'))
    # # pi_net.add(Dense(128, activation='leaky_relu'))
    # pi_net.add(Dropout(0.5))
    # pi_net.add(Dense(64, activation='elu'))
    # pi_net.add(Dropout(0.5))
    # pi_net.add(Dense(32, activation='gelu'))
    # pi_net.add(Dense(4, activation='softmax'))
    # if comp is True:

In [150]:
# 95.83% accuracy
""" pi_net.add(Input(shape=(6,))) 
pi_net.add(Dense(32, activation='leaky_relu'))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(256, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(128, activation='tanh'))
pi_net.add(Dropout(0.5))
pi_net.add(Dense(64, activation='leaky_relu'))
pi_net.add(Dropout(0.25))
pi_net.add(Dense(32, activation='gelu'))
pi_net.add(Dense(16, activation='leaky_relu'))
pi_net.add(Dense(4, activation='softmax')) """

In [ ]:
# 96.15% accuracy 2 wrong
# pi_net.add(Dense(32, activation='silu'))
# pi_net.add(Dense(64, activation='tanh'))
# pi_net.add(Dense(128, activation='leaky_relu'))
# pi_net.add(Dropout(0.5))
# pi_net.add(Dense(64, activation='relu'))
# pi_net.add(Dropout(0.5))
# pi_net.add(Dense(32, activation='gelu'))
# pi_net.add(Dense(4, activation='softmax'))


Keras Classifier

In [123]:
# Use KerasClassifier for scikit-learn compatibility
net = create_pinet(False)
Y_test_pi_labels, Y_pred_pi_labels = train_and_predict_pinets(model=net, opt=optimizers.Adam())
model = KerasClassifier(model=net, epochs=200, batch_size=-1, verbose=1, metrics=['categorical_accuracy'])

# Perform cross-validation
kfold = KFold(n_splits=4, shuffle=True)
results = cross_validate(model, X=pi_groups, y=encoded_y_pi_onehot, cv=kfold, scoring=recall_score(y_true=Y_test_pi_labels, y_pred=Y_pred_pi_labels, average='weighted'))
# recall_score(y_true=Y_test_pi_labels, y_pred=Y_pred_pi_labels, average='weighted')
print(f'Cross-Validation Accuracy: {results.mean():.2f} (+/- {results.std():.2f})')

# train_and_predict_pinets(model=create_pinet(True), opt='adam')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - categorical_accuracy: 0.9849 - loss: 0.0783  
Neural Network Trainset: 
compile_metrics: 97.55%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Testing Confusion Matrix: 
[[20  0  0  2]
 [ 0  7  0  0]
 [ 0  0  3  0]
 [ 1  0  0 19]]
Testing Accuracy: 94.23%
Classification Report
              precision    recall  f1-score   support

           0       0.95      0.91      0.93        22
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         3
           3       0.90      0.95      0.93        20

    accuracy                           0.94        52
   macro avg       0.96      0.96      0.96        52
weighted avg       0.94      0.94      0.94        52



InvalidParameterError: The 'scoring' parameter of cross_validate must be a str among {'roc_auc_ovr', 'positive_likelihood_ratio', 'jaccard_micro', 'neg_brier_score', 'fowlkes_mallows_score', 'balanced_accuracy', 'neg_mean_squared_log_error', 'neg_mean_absolute_error', 'precision_weighted', 'explained_variance', 'jaccard_weighted', 'rand_score', 'jaccard', 'recall_macro', 'adjusted_rand_score', 'roc_auc_ovo_weighted', 'normalized_mutual_info_score', 'f1_samples', 'matthews_corrcoef', 'precision_macro', 'f1_macro', 'neg_root_mean_squared_log_error', 'neg_mean_poisson_deviance', 'precision', 'neg_negative_likelihood_ratio', 'precision_micro', 'roc_auc_ovr_weighted', 'f1_micro', 'recall', 'r2', 'neg_log_loss', 'jaccard_samples', 'f1', 'neg_mean_gamma_deviance', 'f1_weighted', 'precision_samples', 'top_k_accuracy', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'roc_auc', 'completeness_score', 'mutual_info_score', 'neg_mean_squared_error', 'recall_samples', 'homogeneity_score',

Support Vector Machines

In [104]:
clf = svm.SVC()
train_and_predict_using_pimodel('SVM (Classiefier)', clf)


SVM (Classiefier)
Training Confusion Matrix: 
[[72  0  0  6]
 [17  8  0  0]
 [ 5  3  8  0]
 [10  0  0 75]]
Training Accuracy: 79.90%
Testing Confusion Matrix: 
[[20  0  0  2]
 [ 6  1  0  0]
 [ 2  1  0  0]
 [ 4  0  0 16]]
Testing Accuracy: 71.15%


In [105]:
rbf_clf = svm.SVC(kernel='rbf')
train_and_predict_using_pimodel('SVM (Classiefier) - RBF', rbf_clf)

SVM (Classiefier) - RBF
Training Confusion Matrix: 
[[72  0  0  6]
 [17  8  0  0]
 [ 5  3  8  0]
 [10  0  0 75]]
Training Accuracy: 79.90%
Testing Confusion Matrix: 
[[20  0  0  2]
 [ 6  1  0  0]
 [ 2  1  0  0]
 [ 4  0  0 16]]
Testing Accuracy: 71.15%


In [106]:
rbf_svc = svm.SVC(kernel='poly')
train_and_predict_using_model('SVM (Classiefier) - Poly', rbf_svc)

SVM (Classiefier) - Poly
Training Confusion Matrix: 
[[48  0  0 32]
 [24  0  0  0]
 [ 2  0 14  0]
 [17  0  0 67]]
Training Accuracy: 63.24%
Testing Confusion Matrix: 
[[10  0  0 10]
 [ 8  0  0  0]
 [ 1  0  2  0]
 [ 2  0  0 19]]
Testing Accuracy: 59.62%


In [107]:
rbf_svc = svm.SVC(kernel='sigmoid')
train_and_predict_using_model('SVM (Classiefier) - Sigmoid', rbf_svc)

SVM (Classiefier) - Sigmoid
Training Confusion Matrix: 
[[42  0  0 38]
 [11  0  0 13]
 [ 0  0  0 16]
 [50  0  0 34]]
Training Accuracy: 37.25%
Testing Confusion Matrix: 
[[12  0  0  8]
 [ 4  0  0  4]
 [ 0  0  0  3]
 [ 9  0  0 12]]
Testing Accuracy: 46.15%


ValueError: The classes, ['Broken or Intact Sheet', 'Crater', 'Lump', 'Splash'], are not in class_weight